## quick_pp

In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()
all_data = project.get_all_data()

all_data['CPERM'] = all_data.CORE_PERM
all_data['CPORE'] = all_data.CORE_POR / 100

***
# Rock Typing

The process of rock typing is a multidisciplinary effort that involves close collaboration between geologists, petrophysicists, and reservoir engineers. Each discipline plays a specific role, and the rock typing workflow involves a seamless transition of data, models, and insights across these domains to build a robust understanding of the reservoir. Below is the high-level workflow detailing the steps, key tasks, and handoffs between these professionals

Petrophysicists take geological inputs and combine them with well log data and core analysis to define rock types based on petrophysical properties such as porosity, permeability, fluid saturation, and pore geometry.

Tasks:
- Core Data Integration:
    - Perform routine and special core analyses (RCAL, SCAL) to measure porosity, permeability, capillary pressure, and wettability.

- Log Interpretation:
    - Interpret wireline logs (e.g., density, neutron, resistivity, NMR) to generate continuous petrophysical properties along the wellbore.

- Rock Typing Methods:
    - Use statistical tools like Flow Zone Indicator (FZI), Pickett plots, and cluster analysis to group rocks with similar flow behavior into petrophysical rock types.

- Electrofacies Analysis:
    - Identify electrofacies by clustering log responses, especially where core data is limited.

The rock typing workflow is a collaborative process where geologists, petrophysicists, and reservoir engineers work together to ensure the reservoir is accurately characterized and modeled. This interdisciplinary approach ensures that both geological and petrophysical complexities are accounted for, resulting in more efficient production strategies and better field development plans

## Some of the different methods in Rock Typing

As a comparison, the core porosity and permeability are plotted on Winland R35 and Lucia Rock Fabric Number (RFN) methods.

- The plot demonstrates Winland R35 resulted in a less drastic permeability estimation with respect to porosity.
- Lower porosity - high permeability datapoints:
    - These points might be the result of fractured cores.
    - Both Winland R35 and Lucia RFN does not model the datapoints.
    - FZI is able to model but indicates too high of a value.

In [ ]:
from quick_pp.rock_type import plot_fzi, plot_winland, plot_rfn

plot_fzi(all_data['CPORE'], all_data['CPERM'])
plot_winland(all_data['CPORE'], all_data['CPERM'])
plot_rfn(all_data['CPORE'], all_data['CPERM'])

## Winland R35 Rock Typing

In [ ]:
from quick_pp.rock_type import plot_fzi, plot_rfn, plot_winland, rock_typing
from quick_pp.rock_type import calc_r35, rock_typing, plot_winland

# Estimate rock types
r35 = calc_r35(all_data['CPORE'], all_data['CPERM'])
all_data['R35'] = r35
r35_cut_offs = [
    .2, .3, .5, .8, 1.2, 2, 5, 10
]
r35_rock_flag = rock_typing(r35, r35_cut_offs, higher_is_better=True)
all_data['ROCK_FLAG'] = r35_rock_flag

plot_winland(all_data['CPORE'], all_data['CPERM'], rock_type=r35_rock_flag, cut_offs=r35_cut_offs)
print('The data has been classified into the following rock types:')
pd.DataFrame({
    'RRT': r35_rock_flag,
    'r35': r35,
}).reset_index(drop=True).groupby(['RRT']).count()

### Develop Machine Learning models to predict ROCK_FLAG and R35

In [ ]:
from quick_pp.rock_type import train_classification_model, train_regression_model
from sklearn.preprocessing import MinMaxScaler

train_data = all_data.copy()

filters = (
    ~(train_data.WELL_NAME.isin(['HW-3']))  # Filter out wells
    & ~((train_data.CPORE < .02) & (train_data.R35 > 10))  # Filter out low porosity data with high R35
    & (train_data.CPERM > .05)
    & (train_data.R35 > .1)
)
train_data = train_data[filters].copy()

train_data['LOG_RT'] = np.log10(train_data['RT'])
train_data['NDI'] = (2.95 - train_data['RHOB']) / 1.95
train_data['GRN'] = train_data.groupby('WELL_NAME')['GR'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten())

input_features = ['NPHI', 'NDI', 'LOG_RT', 'GRN']
train_data = train_data.dropna(subset=input_features + ['ROCK_FLAG', 'R35'])

r35_rt_model = train_classification_model(
    train_data, input_features=input_features, target_feature='ROCK_FLAG', stratifier=train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\r35_rt_model.qppm', 'wb') as file:
    pickle.dump(r35_rt_model, file)

train_data['LOG_R35'] = np.log10(train_data['R35'])
r35_model = train_regression_model(
    train_data, input_features=input_features, target_feature='LOG_R35', stratifier=train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'wb') as file:
    pickle.dump(r35_model, file)

In [ ]:
all_data['LOG_RT'] = np.log10(all_data['RT'])
all_data['NDI'] = (2.95 - all_data['RHOB']) / 1.95
all_data['GRN'] = all_data.groupby('WELL_NAME')['GR'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten())

predicted_r35_rt = r35_rt_model.predict(all_data[input_features])
plot_winland(all_data['CPORE'], all_data['CPERM'], rock_type=predicted_r35_rt, cut_offs=r35_cut_offs)

In [ ]:
all_data['LOG_RT'] = np.log10(all_data['RT'])
all_data['NDI'] = (2.95 - all_data['RHOB']) / 1.95
all_data['GRN'] = all_data.groupby('WELL_NAME')['GR'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten())

predicted_r35 = 10**(r35_model.predict(all_data[input_features]))
predicted_r35_rt = rock_typing(predicted_r35, r35_cut_offs, higher_is_better=True)
plot_winland(all_data['CPORE'], all_data['CPERM'], rock_type=predicted_r35_rt, cut_offs=r35_cut_offs)

***
# Permeability Transform

### Determining the perm transform parameter for each rock type.

In [ ]:
from ipywidgets import interact, widgets
from quick_pp.core_calibration import poroperm_xplot, fit_poroperm_curve

rt = widgets.Dropdown(
    options=sorted(all_data['ROCK_FLAG'].dropna().unique()),
    value=1,
    description='Rock Type:'
)

@interact(rt=rt)
def param(rt):
    data = all_data[all_data['ROCK_FLAG'] == rt]
    a, b = fit_poroperm_curve(data['CPORE'], data['CPERM'])
    poroperm_xplot(data['CPORE'], data['CPERM'], a=a, b=b, label=rt, log_log=False)

In [ ]:
import pprint

poroperm_params = {}

for rt, data in all_data.groupby('ROCK_FLAG'):
    a, b = fit_poroperm_curve(data['CPORE'], data['CPERM'])
    poroperm_params[rt] = (a, b)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(poroperm_params)

In [ ]:
for rt, data in all_data.groupby('ROCK_FLAG'):
    a, b = poroperm_params[rt]
    poroperm_xplot(data['CPORE'], data['CPERM'], a=a, b=b, label=f'RT{rt}: a={a:1d}, b={b:.2f}')

### Compare different PERM estimations at well level

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

from quick_pp.core_calibration import perm_transform
from quick_pp.rock_type import calc_r35_perm

focus_well = 'HW-26'
well_data = all_data[all_data.WELL_NAME == focus_well].copy()

# Permeability estimation based on core data perm transform
well_data['perm_a'] = well_data['ROCK_FLAG'].map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
well_data['perm_b'] = well_data['ROCK_FLAG'].map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_trans = perm_transform(well_data['PHIT'], well_data['perm_a'], well_data['perm_b'])
perm_r35 = calc_r35_perm(well_data['R35'], well_data['PHIT'])

well_data['LOG_RT'] = np.log10(well_data['RT'])
well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
well_data['GRN'] = MinMaxScaler().fit_transform(well_data[['GR']])

# Permeability prediction based on ROCK_FLAG ML model followed by perm transform
with open(r'data\04_project\MOCK_carbonate\outputs\r35_rt_model.qppm', 'rb') as file:
    r35_rt_model = pickle.load(file)
rock_flag_ml = r35_rt_model.predict(well_data[input_features])
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_ml = perm_transform(well_data['PHIT'], perm_a_ml, perm_b_ml)

# Permeability prediction based on R35 ML model followed by back calculate from R35
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
    r35_model = pickle.load(file)
r35_ml = 10**(r35_model.predict(well_data[input_features]))
perm_r35_ml = calc_r35_perm(r35_ml, well_data['PHIT'])

# Plot to compare
plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, perm_trans, label='Perm Transform')
plt.plot(well_data.DEPTH, perm_r35, label='Perm R35')
plt.plot(well_data.DEPTH, perm_ml, label='Perm ML')
plt.plot(well_data.DEPTH, perm_r35_ml, label='Perm R35 ML')
plt.scatter(well_data.DEPTH, well_data.CPERM, label='Core Perm', marker='.', c='black')
plt.yscale('log')
plt.legend()

# Print scores
score_df = well_data[['DEPTH', 'CPERM']].copy()
score_df['PERM'] = perm_r35_ml
score_df.dropna(inplace=True)
print(f"\n ### PERM MAPE: {mean_absolute_percentage_error(score_df.CPERM, score_df.PERM):.2f}")
print(f" ### PERM R2: {r2_score(score_df.CPERM, score_df.PERM):.2f}")

### Compare different ROCK_FLAG at well level

In [ ]:
# Compare rock types predicted from rt_model with applied cut-offs on predicted r35
rock_flag_r35_ml = rock_typing(r35_ml, r35_cut_offs, higher_is_better=True)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, well_data.ROCK_FLAG, label='Rock Flag Core')
plt.plot(well_data.DEPTH, rock_flag_ml, label='Rock Flag ML')
plt.plot(well_data.DEPTH, rock_flag_r35_ml, label='Rock Flag R35 ML')
plt.legend()


***
# Plotting the result

In [ ]:
from quick_pp.plotter import plotly_log

# Plot individual results
well_data['PERM'] = perm_r35_ml
fig = plotly_log(well_data, 'ft')
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()